## Project #3: Refining Material Design Rules for Good Mg Mobility (Site Volume)
Undergraduate Researcher: Tasha Lewis

* Combination of coding to characterize materials and analyze a preliminary data set
* Develop code for evaluating insertion site volume and study the relationship with Mg mobility
* Short term goal: Use 2 materials for code development
* Next steps: Apply code to ApproxNEB data set of 118 Mg cathodes and analyze preliminary data 

Primary Mentor: Qian, Secondary Mentor: Ann

Funded as needed by LBNL: Joint Center for Energy Storage Research - Charge Transfer (JCESR 2.0 CT)

In [1]:
from maggma.stores.advanced_stores import MongograntStore
from maggma.stores.compound_stores import ConcatStore

/Users/acrutt/miniconda3/envs/ugr/lib/python3.9/site-packages/maggma/utils.py:20: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
acrt_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","tasks",key="task_id")
acrt_store.connect()

jst_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","js_tasks",key="task_id")
jst_store.connect()

tasks_store = ConcatStore(stores=[acrt_store,jst_store],key="task_id")
tasks_store.connect()

## Data for Development

Open question: What structure is best to look at for using site volume as a predictor? Relaxed Mg inserted unit cell structure? Unrelaxed Mg inserted unit cell structure? Migration graph structure with all inserted sites filled? Can we get away with the relaxed empty base unit cell structure to avoid insertion calculations?

* **Sodiated Birnessite NaMn4O8:** "Good" larger site volume
    * base task_ids: js-46421
    * Mg insertions task_ids: js-119679, js-119666
    * batt_id: "119666_Mg"
    * approx_neb wf_uuid: "9da400ce-b10e-496b-89f4-404c7cda55bb"
    * empty host mp_id: mp-1016155
* **Spinel MnO2:** "Bad" smaller site volume
    * base task_ids: js-18
    * Mg insertions task_ids: js-72614, js-113091, js-113096, js-47836, js-47314, js-46959, js-46581
    * batt_id: "113091_Mg" (approx_neb), "128543_Mg" (electrode)
    * approx_neb wf_uuid: "938826bf-676d-473e-b98f-0c3c9f842eed"
    * empty host mp_id: mp-25275

## Verification Data

More cases for confirming analysis works as expected.

* **Spinel TiS2:** "Good" larger site volume
    * base task_ids: 4527
    * Mg insertions task_ids: 4702, 4704, 5246
    * batt_id: "spinelTi2S4_Mg" (approx_neb)
    * approx_neb wf_uuid: 
    * empty host mp_id: mp-9027
* **NASICON V2(PO4)3:** "Good" larger site volume
    * base task_ids: 8772
    * Mg insertions task_ids: 14352, 14950, 15209
    * batt_id: "43005_Mg", "NVP_Mg" (approx_neb)
    * approx_neb wf_uuid: "4599dc51-efd2-4b32-ac51-0cc982f08bcd"
    * empty host mp_id: mp-26962
* **Tavorite MnPO4F:** "Bad" smaller site volume
    * base task_ids: js-47435
    * Mg insertions task_ids: js-128543, js-128568
    * batt_id: "128543_Mg"
    * approx_neb wf_uuid: "7bd62bf7-a09c-4bf0-a02e-1224e00663b2"
    * empty host mp_id: mp-25426

## Validation Data

Preliminary data set of 118 Mg cathodes to evaluate the value of site volume as a predictor.

Battery IDs are stored in `prelim_data_118.json`
* 78 prioritized from 230 Mg electrodes selected from screening. Note these batches include some materials outside the prioritized 78.
    * 14 from "20191122_batch"
    * 76 from "20200214_batch" 
    * 18 from "20200406_batch"
* 9 sulfides ("20200410_batch" tag)
* 41 thiophosphates ("20201106_batch" tag)

Get relevant (insertion workflow) unit cell calculations from task_ids specified in electrode docs matching these battery ids

In [3]:
elec_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","vw_elec",key="battid")
elec_store.connect()

In [4]:
aneb_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","approx_neb",key="wf_uuid")
aneb_store.connect()

aneba_store = MongograntStore("ro:mongodb07-ext.nersc.gov/fw_acr_mv","approx_neb_analysis",key="wf_uuid")
aneba_store.connect()

In [5]:
aneb_store.distinct("batt_id",{"tags":{"$in":["20200410_batch"]}}) 

['113245_Mg',
 '114444_Mg',
 '119311_Mg',
 '126477_Mg',
 '127214_Mg',
 '128032_Mg',
 '132383_Mg',
 '14248_Mg',
 '17332_Mg']

In [6]:
aneb_store.distinct("batt_id",{"tags":{"$in":["20201106_batch"]}}) 

['6765_Mg',
 '6766_Mg',
 '6768_Mg',
 '6769_Mg',
 '6771_Mg',
 '6772_Mg',
 '6776_Mg',
 '6777_Mg',
 '6779_Mg',
 '6781_Mg',
 '6783_Mg',
 '6784_Mg',
 '6785_Mg',
 '6789_Mg',
 '6790_Mg',
 '6792_Mg',
 '6794_Mg',
 '6796_Mg',
 '6798_Mg',
 '6801_Mg',
 '6806_Mg',
 '6807_Mg',
 '6809_Mg',
 '6811_Mg',
 '6813_Mg',
 '6818_Mg',
 '6820_Mg',
 '6821_Mg',
 '6824_Mg',
 '6827_Mg',
 '6845_Mg']

## Background Reading

https://doi.org/10.1021/acs.chemmater.5b02342: summarizes understanding of multivalent ion mobility focused on coordination

https://arxiv.org/abs/2204.05383: introduces hypothesis for considering site volume in addition to coordination